# Run the `Heat` model through its BMI

`Heat` models the diffusion of temperature on a uniform rectangular plate with Dirichlet boundary conditions. View the source code for the [model](https://github.com/csdms/bmi-example-python/blob/master/heat/heat.py) and its [BMI](https://github.com/csdms/bmi-example-python/blob/master/heat/bmi_heat.py) on GitHub.

Start by importing `os`, `numpy` and the `Heat` BMI:

In [26]:
import os
import numpy as np
import pandas as pd

from heat import BmiHeat

Create an instance of the model's BMI.

In [2]:
x = BmiHeat()

What's the name of this model?

In [3]:
print(x.get_component_name())

The 2D Heat Equation


Start the `Heat` model through its BMI using a configuration file:

In [4]:
cat heat.yaml

# Heat model configuration
shape:
  - 6
  - 8
spacing:
  - 1.0
  - 1.0
origin:
  - 0.0
  - 0.0
alpha: 1.0


In [5]:
x.initialize("heat.yaml")

Check the time information for the model.

In [6]:
stateOut = x.get_state()
print(stateOut)

{
    "time": 0.0,
    "plate_surface__temperature": {
        "value": [
            0.7751559929795838,
            0.9740429959945077,
            0.07602360261172714,
            0.19345617181867514,
            0.39467485155774207,
            0.6005565853122184,
            0.3229424519769787,
            0.987630806490736,
            0.3874355285568958,
            0.6700808824401435,
            0.8228438420280402,
            0.7460197054142051,
            0.6105962442057601,
            0.9683361960938874,
            0.7873742592700971,
            0.8297664301888102,
            0.09310980618841402,
            0.28162754601767714,
            0.4171203350603051,
            0.5738856325374869,
            0.803987360854201,
            0.34090547980542085,
            0.4776026525000646,
            0.987316125397035,
            0.8842297614848268,
            0.6417553192113641,
            0.7861712685558294,
            0.20659033047583297,
            0.072281219623

In [7]:
import json
stateOutDict = json.loads(stateOut)

In [8]:
stateOutDict['plate_surface__temperature'][15]=0
print(stateOutDict)

{'time': 0.0, 'plate_surface__temperature': {'value': [0.7751559929795838, 0.9740429959945077, 0.07602360261172714, 0.19345617181867514, 0.39467485155774207, 0.6005565853122184, 0.3229424519769787, 0.987630806490736, 0.3874355285568958, 0.6700808824401435, 0.8228438420280402, 0.7460197054142051, 0.6105962442057601, 0.9683361960938874, 0.7873742592700971, 0.8297664301888102, 0.09310980618841402, 0.28162754601767714, 0.4171203350603051, 0.5738856325374869, 0.803987360854201, 0.34090547980542085, 0.4776026525000646, 0.987316125397035, 0.8842297614848268, 0.6417553192113641, 0.7861712685558294, 0.20659033047583297, 0.07228121962360368, 0.7885444991267336, 0.1582292399062314, 0.7442878504063878, 0.07447221802960291, 0.5780722851775619, 0.6943635604104812, 0.2821665942733348, 0.4685240374159698, 0.10018338876202237, 0.04591115958116665, 0.7646718853795121, 0.26417852319698265, 0.8745077022566503, 0.9655401571468339, 0.041403279912340385, 0.33964789534654916, 0.8611199139560012, 0.54687791302

In [9]:
x.set_state(json.dumps(stateOutDict))

In [10]:
print("Start time:", x.get_start_time())
print("End time:", x.get_end_time())
print("Current time:", x.get_current_time())
print("Time step:", x.get_time_step())
print("Time units:", x.get_time_units())

Start time: 0.0
End time: 1.7976931348623157e+308
Current time: 0.0
Time step: 0.25
Time units: s


Show the input and output variables for the component (aside on [Standard Names](https://csdms.colorado.edu/wiki/CSDMS_Standard_Names)):

In [11]:
print(x.get_input_var_names())
print(x.get_output_var_names())

('plate_surface__temperature',)
('plate_surface__temperature',)


Next, get the identifier for the grid on which the temperature variable is defined:

In [12]:
grid_id = x.get_var_grid("plate_surface__temperature")
print("Grid id:", grid_id)

Grid id: 0


Then get the grid attributes:

In [13]:
print("Grid type:", x.get_grid_type(grid_id))

rank = x.get_grid_rank(grid_id)
print("Grid rank:", rank)

shape = np.ndarray(rank, dtype=int)
x.get_grid_shape(grid_id, shape)
print("Grid shape:", shape)

spacing = np.ndarray(rank, dtype=float)
x.get_grid_spacing(grid_id, spacing)
print("Grid spacing:", spacing)

Grid type: uniform_rectilinear
Grid rank: 2
Grid shape: [6 8]
Grid spacing: [1. 1.]


These commands are made somewhat un-Pythonic by the generic design of the BMI.

Through the model's BMI, zero out the initial temperature field, except for an impulse near the middle.
Note that *set_value* expects a one-dimensional array for input.

In [14]:
temperature = np.zeros(shape)
temperature[3, 4] = 100.0
x.set_value("plate_surface__temperature", temperature)

Check that the temperature field has been updated. Note that *get_value* expects a one-dimensional array to receive output.

In [15]:
temperature_flat = np.empty_like(temperature).flatten()
x.get_value("plate_surface__temperature", temperature_flat)
print(temperature_flat.reshape(shape))

[[  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0. 100.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.]]


Now advance the model by a single time step:

In [16]:
x.update()

View the new state of the temperature field:

In [17]:
x.get_value("plate_surface__temperature", temperature_flat)
print(temperature_flat.reshape(shape))

[[ 0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.  12.5  0.   0.   0. ]
 [ 0.   0.   0.  12.5 50.  12.5  0.   0. ]
 [ 0.   0.   0.   0.  12.5  0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0. ]]


There's diffusion!

## Generate an ensemble of models using get_state and set_state
We will demonstrate (some of) the use of get_state and set_state by generating an ensemble of models starting from the state of the one we just initialized, adding noise to each ensemble member and than running all of them forward in time. While running forward, we save the temperature at a single point of interest and plot a graph of all ensemble members to show the ensemble spread over time.

First, we save the current state of the model.

In [18]:
state_out = x.get_state()

State is always returned as a string. In this case the string is formated as JSON, so Let's see what keys are in this state:

In [19]:
dictState = json.loads(state_out)
print(dictState.keys())

dict_keys(['time', 'plate_surface__temperature'])


In [20]:
print(dictState['plate_surface__temperature'].keys())

dict_keys(['value', 'type', 'itemsize', 'nbytes'])


Next, we create an ensemble of BMI model objects. Each model gets initzialized in the same way as ```x``` was above.

In [21]:
nEnsemble = 25
ensemble = []

for ensembleMember in range(nEnsemble):
    ensemble.append(BmiHeat())
    ensemble[ensembleMember].initialize("heat.yaml")

Now, for each ensemble member, we take the state of x, add some noise, and set the state of the member

In [25]:
plate_surface_temp_from_x = dictState['plate_surface__temperature']['value']

for ensembleMember in range(nEnsemble):
    #for each ensemble, start with the same state as x.
    ensembleMemberDictState = dictState
    
    noise = np.random.randn(len(ensembleMemberDictState['plate_surface__temperature']['value']))
    plate_surface_temp = plate_surface_temp_from_x + noise
    
    #this array we want to set in the state is a numpy array, which has to be transformed 
    #to a list otherwise we can not turn it into a json string.
    ensembleMemberDictState['plate_surface__temperature']['value'] = plate_surface_temp.tolist()
    
    #Turn the state with noise added back into a json string
    ensembleStateJSON = json.dumps(ensembleMemberDictState)
    
    #finally set the state of the ensemble member.
    ensemble[ensembleMember].set_state(ensembleStateJSON)
    
#Note that the above loop is needlessly verbose for educational purposes.
#This does use a lot of memmory. The code below is less readable, but uses less memmory.
#When using bigger models than this example model, this can make a big difference in 
#performance.

plate_surface_temp_from_x = dictState['plate_surface__temperature']['value']

for ensembleMember in range(nEnsemble):

    dictState['plate_surface__temperature']['value'] = (plate_surface_temp_from_x + 
                                                        np.random.randn(len(ensembleMemberDictState['plate_surface__temperature']['value']))).tolist()
    ensemble[ensembleMember].set_state(json.dumps(dictState))


Now run the entire ensemble forward in time. Every timestep, we are saving the temperature of one location of the plate of interest, for each ensemble member.

Note that for bigger models, the for loop in this step can be run in parallel for all the models. 

In [50]:
loc_of_interest = np.array([3,4])

distant_time = 2.0
output = pd.DataFrame(columns = ['x'])

outputValue = np.empty([0])
while x.get_current_time() < distant_time:
    x.update()
    x.get_value_at_indices('plate_surface__temperature',outputValue, loc_of_interest)
    output.loc[x.get_current_time(),'x'] = outputValue
    
    for ensembleMember in range(nEnsemble):
        ensemble[ensembleMember].update()
        ensemble[ensembleMember].get_value_at_indices('plate_surface__temperature',outputValue, loc_of_interest)
        output.loc[ensemble[ensembleMember].get_current_time(),'ensemble' + str(ensembleMember)] = outputValue
        


ValueError: could not broadcast input array from shape (2) into shape (0)

View the final state of the temperature field:

In [49]:
ensemble[ensembleMember].get_value_at_indices('plate_surface__temperature',outputValue, loc_of_interest)

array([], dtype=float64)

In [ ]:
np.set_printoptions(formatter={"float": "{: 5.1f}".format})
x.get_value("plate_surface__temperature", temperature_flat)
print(temperature_flat.reshape(shape))

Note that temperature isn't conserved on the plate:

In [ ]:
print(temperature_flat.sum())

End the model:

In [ ]:
x.finalize()